# Работа с базами данных

Материалы:
* Макрушин С.В. Лекция 6: Работа с базами данных
* https://sqliteonline.com/
* https://docs.python.org/3/library/sqlite3.html
* https://www.geeksforgeeks.org/sql-join-set-1-inner-left-right-and-full-joins/
* https://www.datacamp.com/community/tutorials/group-by-having-clause-sql

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Задачи для совместного разбора

1. Работая с базой данных `Chinook_Sqlite.sqlite`, найдите и выведите на экран имена и фамилии всех заказчиков из Канады

In [ ]:
import sqlite3

con = sqlite3.connect('/content/drive/MyDrive/Colab Notebooks/lab 6/Chinook_Sqlite.sqlite')
cur = con.cursor()

cur.execute('''
SELECT firstname, lastname
FROM Customer 
WHERE country=?
''', ('Canada', ))

print(cur.fetchmany(5))

[('François', 'Tremblay'), ('Mark', 'Philips'), ('Jennifer', 'Peterson'), ('Robert', 'Brown'), ('Edward', 'Francis')]


2. Найти и вывести на экран названия всех альбомов группы Accept

3. Создайте базу данных с названием вашей группы. В этой базе данных создайте таблицу Student, содержащую 2 столбца: id и name. Добавьте в таблицу Student информацию о студентах, сидящих с вами по соседству.

## Лабораторная работа 6

1. Создайте файл базы данных c названием `recipes.db`. Создайте объект-курсор. 

In [1]:
import sqlite3

con = sqlite3.connect('recipes.db')
cur = con.cursor()



2. Напишите и выполните SQL-запрос для создания таблицы рецептов `Recipe`. Таблица должна содержать следующие поля:
`id`, `name`, `minutes`, `submitted`, `description`, `n_ingredients`. Определитесь с типами и составом ключевых полей.

In [14]:
def droppp (cur):
  pass
  #cur.execute("""DROP TABLE IF EXISTS Recipe;""")

def create_table(cur, create_table_sql):
  cur.execute(create_table_sql)

message="""
                                      CREATE TABLE Recipe (
                                        id integer PRIMARY KEY,
                                        name text,
                                        minutes integer,
                                        submitted DATETIME,
                                        description text,
                                        n_ingredients integer
                                    ); """

droppp(cur)
create_table (cur, message)

3. Напишите и выполните SQL-запрос для создания таблицы отзывов `Review`. Таблица должна содержать следующие поля:
`id`, `user_id`, `recipe_id`, `date`, `rating`, `review`. Определитесь с типами полей, набором ключевых полей. При помощи внешнего ключа соедините две таблицы.

In [32]:
def droppp (cur):
  #pass
  cur.execute("""DROP TABLE IF EXISTS Review;""")

def create_table(cur, create_table_sql):
  cur.execute(create_table_sql)

message="""
                                      CREATE TABLE Review (
                                        id integer PRIMARY KEY,
                                        user_id integer,
                                        date DATETIME,
                                        rating integer,
                                        review text,
                                        recipe_id integer  UNIQUE,
                                        FOREIGN KEY (recipe_id) REFERENCES Recipe (id)

                                    ); """

droppp(cur)
create_table (cur, message)

4. Загрузите данные из файлов `reviews_sample.csv` (__ЛР2__) и `recipes_sample_with_tags_ingredients.csv` (__ЛР5__) в созданные таблицы

In [36]:
import pandas as pd
import csv
from sqlalchemy.types import Integer

reviews = pd.read_csv ('/content/drive/MyDrive/Colab Notebooks/lab 3/reviews_sample.csv' , names=['id', 'user_id','recipe_id', 'date','rating', 'review'  ] , sep=',')
recipes = pd.read_csv ('/content/drive/MyDrive/Colab Notebooks/recipes_sample_with_filled_nsteps.csv', sep=',' , usecols=['id', 'name', 'minutes', 'submitted','description', 'n_ingredients'] )


#cur.executemany("INSERT INTO Review VALUES (?, ?, ? , ? , ?, ?)", reviews.values)


#cur.executemany("INSERT INTO Recipe VALUES (?, ?, ? , ? , ?, ?)", recipes.values)



con.commit()


#cur.execute("SELECT * FROM Review")



Index(['id', 'name', 'minutes', 'submitted', 'description', 'n_ingredients'], dtype='object')
[67664 'healthy for them  yogurt popsicles' 10 '2003-07-26'
 'my children and their friends ask for my homemade popsicles morning, noon and night. i never turn them down; who am i to tell them that they are good for them! for variety i substitute different flavours of frozen juice - grape, fruit punch, tropical etc.'
 nan]


5. Найдите все рецепты, для выполнения которых нужно ровно 10 ингредиентов. Выведите на экран первые 5 из найденных рецептов.

In [37]:

cur.execute("SELECT * FROM Recipe WHERE n_ingredients =10.0 LIMIT 5")

rows = cur.fetchall()

for row in rows:
  print(row)

(246, 'lee s hot crab dip', 45, '1999-09-01', "lee's hot crab dip", 10)
(289, 'feijoada  brazilian bean soup  ii', 150, '1999-10-04', 'brazilian bean soup', 10)
(544, 'greek mushroom salad', 38, '1999-08-10', None, 10)
(671, 'cucumber relish', 40, '1999-09-10', None, 10)
(975, 'basil parmesan biscuits', 27, '1999-08-14', 'perfect with pasta or other italian dishes', 10)


6. Найдите название рецепта, для выполнения которого требуется больше всего времени.

In [41]:
cur.execute("SELECT name  FROM Recipe order by CAST(minutes as numeric) DESC LIMIT 1")
#cur.execute('PRAGMA table_info(Recipe);')
rows = cur.fetchall()

for row in rows:
  print(row)

('strawberry liqueur',)


7. Запросите у пользователя id рецепта и верните информацию об этом рецепте. Если рецепт отсуствует, выведите соответствующее сообщение.

In [133]:
numb = int(input("Abobus, please enter your ID "))

def output(cur, command):
  cur.execute(command)

command = f""
#output(cur, command)

Abobus, please enter your ID 23


8. Найдите кол-во отзывов с рейтингом 5.

9. Найдите кол-во уникальных рецептов, не имеющих отзывов с рейтингом, меньше 4. 

10. Найдите кол-во рецептов, опубликованных в 2010 году и имеющих длину не менее 15 минут.

11. Выберите id рецепта, название рецепта, id пользователя, оставившего отзыв, дату отзыва и рейтинг для тех рецептов, которые имеют не менее 3 ингредиентов. Отсортируйте результат по id рецепта.